In [ ]:
#| default_exp model.rpn

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
import cv2
import timm
import torch
import pickle
import random
import numpy as np
import torch.nn as nn
from fastai.vision.all import *
from license_plate_detection.utils import *

In [ ]:
#| export
class AnchorGenerator:
    def __init__(self, 
                 stride, 
                 ratios, 
                 scales, 
                 ft_shape, 
                 device):
        self.device=device
        self.stride=np.array(stride).ravel()
        self.ratios=np.array(ratios)
        self.scales=np.array(scales)
        self.ft_shape=ft_shape
        
    def generate_base_anchors(self):
        h_ratios= np.sqrt(self.ratios)
        w_ratios=1/h_ratios
        hs =(h_ratios[:, None]*self.scales[None, :]).ravel()
        ws =(w_ratios[:, None]*self.scales[None, :]).ravel()
        xc, yc= 0., 0.
        base_anchors=np.array([
            xc-0.5*ws, yc-0.5*hs, xc+0.5*ws, yc+0.5*hs
        ]).T
        return base_anchors
    
    def generate_all_anchors(self):
        base_anchors= self.generate_base_anchors()
        num_k= base_anchors.shape[0]
        h, w = self.ft_shape
        shift_x= np.arange(0, w)*self.stride[1]
        shift_y= np.arange(0, h)*self.stride[0]
        shift_xx, shift_yy= np.meshgrid(shift_x, shift_y)
        shifts= np.stack([shift_xx.ravel(), shift_yy.ravel(), shift_xx.ravel(), shift_yy.ravel()])
        all_anchors= base_anchors[None, :, :]+ shifts.T[:, None, :]
        all_anchors= all_anchors.reshape(-1, 4)
        all_anchors= torch.Tensor(all_anchors).to(self.device)
        return all_anchors, num_k

In [ ]:
#| export
class RPN(nn.Module):
    def __init__(self, 
                 in_channels,
                 stride, 
                 ratios, 
                 scales, 
                 ft_shape, 
                 device,
                 nms_pre=1000, 
                 nms_post=100, 
                 nms_thr=0.5):
        super().__init__()
        
        self.all_anchors, self.num_k=self.generate_all_anchors(stride,ratios,scales,ft_shape,device) 
        self.shared_conv=ConvLayer(in_channels,in_channels//2,3,1,1,xtra=nn.Dropout())
        self.reg_conv= nn.Conv2d(in_channels//2,4*self.num_k,1,1,0)
        self.cls_conv= nn.Conv2d(in_channels//2,2*self.num_k,1,1,0)
        self.nms_pre= nms_pre
        self.nms_post= nms_post
        self.nms_thr= nms_thr
    
    def generate_all_anchors(self, 
                       stride, 
                       ratios, 
                       scales, 
                       ft_shape, 
                       device):
        
        generator= AnchorGenerator(stride, 
                                   ratios, 
                                   scales, 
                                   ft_shape,
                                   device)
        return generator.generate_all_anchors()
    
    
    def get_rpn_gt(self, gt_bboxes, size):
        gt_rpn_cls=[]
        gt_rpn_reg=[]
        all_anchors= self.all_anchors
        for box in gt_bboxes:
            gt_bboxes= unscale_pnts(box, size)
            valid_flags= assign_anchor_torch(all_anchors, 
                                             gt_bboxes.unsqueeze(0), 
                                             pos_iou_thr=0.5, 
                                             neg_iou_thr=0.2)
            rpn_cls= sample_anchor_torch(all_anchors, 
                                         valid_flags, 
                                         num_per_img=5)
            rpn_reg= calc_offset_torch(all_anchors,gt_bboxes)
            gt_rpn_cls.append(rpn_cls)
            gt_rpn_reg.append(rpn_reg)
        return torch.stack(gt_rpn_cls), torch.stack(gt_rpn_reg)

    def get_rois(self, reg_feats, cls_feats):
        fg_score, fg_ind= cls_feats[:,:,1].sort(descending=True)
        all_rois=[]
        for b_idx,_ in enumerate(fg_ind):
            top_ind= fg_ind[b_idx,:self.nms_pre]
            top_score= fg_score[b_idx,:self.nms_pre]
            top_reg_feats= reg_feats[b_idx,top_ind,:]
            top_anchors= self.all_anchors[top_ind]
            bboxes= reverse_offset(top_anchors, top_reg_feats)
            bboxes_nms= torch.hstack((bboxes, top_score.unsqueeze(1)))
            # bboxes_nms= nms(bboxes, self.nms_thr)[:self.nms_post]
            roi_indices= torch.ones((bboxes_nms.shape[0], 1))*b_idx
            roi_indices= check_device(roi_indices, bboxes_nms.device)
            rois= torch.cat((roi_indices, bboxes_nms[:, :-1]), dim=1)
            all_rois.append(rois)
        return torch.stack(all_rois)
        
    def forward(self, x):
        x= self.shared_conv(x)
        reg_feats= self.reg_conv(x)
        cls_feats= self.cls_conv(x)
        reg_feats= reg2loc(reg_feats)
        cls_feats= cls_reshape(cls_feats)
        cls_feats= F.softmax(cls_feats, dim=2)
        rois= self.get_rois(reg_feats, cls_feats)
        return reg_feats, cls_feats, rois

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()